In [0]:
USE CATALOG crowd2fund;

In [0]:
CREATE OR REFRESH STREAMING TABLE bronze.investments
COMMENT 'Investment Data imported from Crowd2Fund export'
TBLPROPERTIES ('quality' = 'bronze','delta.columnMapping.mode' = 'name')
AS
SELECT *,
      _metadata.file_path AS file_path 
  FROM STREAM read_files('/Volumes/crowd2fund/landing/exports/investments',
                  format => 'csv',
                  delimiter => ',',
                  header => true);

In [0]:
CREATE OR REFRESH PRIVATE MATERIALIZED VIEW v_business_transactions AS
SELECT t.Date,
       t.Time,
       t.Amount_In_GBP,
       t.Amount_Out_GBP,
       i.`Business name` as Business_Name
FROM bronze.transactions t
INNER JOIN bronze.investments i ON t.Date = i.`Investment date` AND t.Amount_Out_GBP = i.`Invested (GBP)`
WHERE t.From_To LIKE 'Exchange with%'
  AND t.Amount_Out_GBP IS NOT NULL
  AND i.`Invested (GBP)` IS NOT NULL
UNION ALL
SELECT t.Date,
       t.Time,
       t.Amount_In_GBP,
       t.Amount_Out_GBP,
       t.From_To as Business_Name
FROM bronze.transactions t
WHERE NOT t.From_To LIKE 'Exchange with%'
  AND t.From_To <> 'Add Funds'
  AND t.From_To <> 'Withdrawal Request'

In [0]:
CREATE OR REFRESH PRIVATE MATERIALIZED VIEW v_businesses AS
SELECT
  crowd2fund.default.get_basename(Business_name) AS basename,
  max_by(Business_name, length(Business_name)) AS Business_name
FROM
  v_business_transactions
GROUP BY basename

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW silver.business_transactions2 AS
SELECT bt.Date, bt.Time, bt.Amount_In_GBP, bt.Amount_Out_GBP, b.Business_Name
  FROM v_business_transactions bt
  INNER JOIN v_businesses b ON crowd2fund.default.get_basename(bt.Business_name) = b.basename